In [1]:
import json
from collections import defaultdict
import re
import random
import string
import math
import numpy as np
from operator import itemgetter 


In [2]:
recipes = []
roles = defaultdict(set)

with open('./data.json') as json_file:
    data = json.load(json_file)
    for p in data:
        recipes.append(p)
        for ing in p['ingredients']:
            if 'role' not in ing: # remove this
                continue
            roles[ing['role']].add(ing['name'])


In [26]:
cup_equivalent = {
    'cup':1.0,
    'cups':1.0,
    'package':3.0,#random
    'packages': 3.0,
    'box': 4.0,#random
    'boxes': 4.0,
    'tablespoon':0.0625,
    'tablespoons':0.0625,
    'teaspoon': 0.02,
    'teaspoons': 0.02,
    'pound':3.5,
    'pounds':3.5,
    'pint':2.0,
    'pints':2.0,
    'jar':2.0,
    'jars':2.0,
    'stick':0.5,
    'sticks':0.5,
    'quart': 4,
    'quarts':4,
    'ounce':0.125,
    'ounces':0.125,
    'fluid': 0.125, #fluid ounces
    'fluids':0.125,
    'units':1.0,
}
def quantity_norm(ingredient):
    if ingredient['measure'] not in cup_equivalent:
        print(ingredient)
    return float(ingredient['quantity']) * cup_equivalent[ingredient['measure']]

1. The rule of thumb for cookie baking is to always keep the amount of fat and sugar used relatively equal. There should be less than 1/4 cup difference between the two.
2. The amount of flour used should be about two times the amount of fat. 
3. If you use baking soda, your recipe must include another acidic ingredient, like sour cream, lemon juice, or buttermilk.
4. Number of flavoring ingredients
5. How uncommon is an ingredient

In [62]:
def total_role(ingredients,role):
    total = 0
    for ingredient in ingredients:
        if ingredient['role'] == role:
            total += quantity_norm(ingredient)
    return total
def fitness(recipe):
    ingredients = recipe['ingredients']
    total_sugar = total_role(ingredients,'sugar')
    total_fat = total_role(ingredients,'fat')
    total_flour = total_role(ingredients,'flour')
    
    fat_sugar_balance = 0 if total_sugar+total_fat == 0 else (1 - abs(total_sugar-total_fat)/(total_sugar+total_fat))       #1
    fat_sugar_flour = 0 if total_flour+total_fat == 0 else (1 - abs(total_flour-total_fat * 2)/(total_flour+total_fat * 2)) #2
    fitness_score = fat_sugar_balance +  fat_sugar_flour
    #print(fitness_score)
    #print(total_sugar,total_fat,total_flour,fat_sugar_balance,fat_sugar_flour)
    return fitness_score


In [63]:
#fitness(recipes[2])
for recipe in recipes:
    fitness(recipe)

In [ ]:

# todo: quantities are missing
def crossover(recipe1, recipe2):
    ingredients1 = defaultdict(list)
    ingredients2 = defaultdict(list)
    roles = set()
    
    for ingredient in recipe1['ingredients']:
        roles.add(ingredient['role'])
        ingredients1[ingredient['role']].append(ingredient['name'])

    for ingredient in recipe2['ingredients']:
        roles.add(ingredient['role'])
        ingredients2[ingredient['role']].append(ingredient['name'])


    new_recipe = {}
    new_recipe['title'] = ''.join(random.choice(string.ascii_letters) for i in range((int( (len(recipe1['title']) + len(recipe2['title']))/2))))
    new_recipe['ingredients'] = []
    for role in roles:  
        nr1 = len(ingredients1[role])
        nr2 = len(ingredients2[role])
        
        if nr1 == 1 and nr2 == 1: #choose from one group
            r1 = random.randint(0, 1)
            if r1 == 0:
                new_recipe['ingredients'].append({'name': ingredients1[role][0], 'role' : role})
            else :
                new_recipe['ingredients'].append({'name': ingredients1[role][0], 'role' : role})

        elif nr1 + nr2 == 1:
            merged = ingredients1[role] + ingredients2[role]
            new_recipe['ingredients'].append({'name': merged[0], 'role' : role})

        else: # take more and less half of each
            indexes1= np.random.randint(0,nr1 ,math.ceil(nr1/2))
            l1 = list(map(ingredients1[role].__getitem__, indexes1))
            indexes2= np.random.randint(0,nr2 ,math.ceil(nr1/2))
            l2 = list(map(ingredients2[role].__getitem__, indexes2))
            s = set(l1 + l2)
            for ing in list(s):
                new_recipe['ingredients'].append({'name': ing, 'role' : role})
    
    return new_recipe

new_recipe = crossover(recipes[40], recipes[2])


In [ ]:
def mutation(recipe):
    # remove randomly an element:
    r1 = random.randint(0, len(recipe['ingredients'])-1)
    recipe['ingredients'].pop(r1)

    # add randomly an ingredient:
    role, ingredients = random.choice(list(roles.items()))
    ingredient = random.choice(tuple(ingredients))
    recipe['ingredients'].append({'name' : ingredient, 'role': role})

    # todo: change quantities

    # replace by other with the same role:
    role = recipe['ingredients'][r1]['role']
    r2 = random.randint(0, len(roles[role])-1)
    recipe['ingredients'][r1]['name'] = list(roles[role])[r2]
    return recipe


print(new_recipe)
mutation(new_recipe)

{'title': 'KArqlFmKYQqUeDgnZNthQPBTMqwxzwAnF', 'ingredients': [{'name': 'butter', 'role': 'fat'}, {'name': 'all-purpose flour', 'role': 'flour'}, {'name': 'egg yolk', 'role': 'binding agent'}, {'name': 'egg yolks', 'role': 'binding agent'}, {'name': 'white sugar', 'role': 'sugar'}, {'name': 'baking soda', 'role': 'rising agent'}, {'name': 'fruit preserves', 'role': 'flavor texture'}, {'name': 'vanilla extract', 'role': 'flavor texture'}]}


{'title': 'KArqlFmKYQqUeDgnZNthQPBTMqwxzwAnF',
 'ingredients': [{'name': 'butter', 'role': 'fat'},
  {'name': 'all-purpose flour', 'role': 'flour'},
  {'name': 'egg yolk', 'role': 'binding agent'},
  {'name': 'egg yolks', 'role': 'binding agent'},
  {'name': 'white sugar', 'role': 'sugar'},
  {'name': 'baking soda', 'role': 'rising agent'},
  {'name': 'ground pistachios', 'role': 'flavor texture'},
  {'name': 'quick rise yeast', 'role': 'rising agent'}]}

In [ ]:

recipes = []

graph = defaultdict(list)
replacements = []

ingredient_bag = defaultdict(int)


def parse_ingredient(line):

    line = line.lstrip()
    idx = 0
    # devemos tentar aqui separar as unidades ou simplesmente unidades + elementos são o proprio elemento?
    #while (line[idx].isdigit() or line[idx] == '/' or line[idx] == ' '):
    #    idx+=1
    
    #quantity = line [0 : idx].lstrip()
    
    ingredients = line[idx:].lower()

    ingredients = ingredients.split(' or ')

    for i in range(len(ingredients)):
        ingredients[i] = ingredients[i].lstrip()

    return quantity, ingredients

with open('./data.json') as json_file:
    data = json.load(json_file)
    for p in data:
        recipes.append(p)
        for i in range (len(p['ingredients']) - 1):
            for j in range( i + 1, len(p['ingredients'])):

                quantity, ingredients = parse_ingredient(p['ingredients'][i])
                quantity2, ingredients2 = parse_ingredient(p['ingredients'][j]) 

                if len(ingredients) > 1:
                    replacements.append(ingredients)

                if len(ingredients2) > 1:
                    replacements.append(ingredients2)

                for ing in ingredients:
                    graph[ing] += ingredients2
                    ingredient_bag[ing]+=1

                for ing in ingredients2:
                    graph[ing] += ingredients
                    ingredient_bag[ing]+=1
                


for k, v in ingredient_bag.items():
    print("{}->{}".format(k, v))
#for n in graph:
#    print("{} -> {}".format(n , len(graph[n])))

AttributeError: 'dict' object has no attribute 'lstrip'

In [ ]:
from queue import PriorityQueue

def calculate_distance(graph, ingredient):
    pq = PriorityQueue()
    dist = defaultdict(int)

    pq.put((0, ingredient ))
    dist[ingredient] = 0
    visited = set()

    while not pq.empty():
        u = pq.get()[1]
        for i in graph[u]:
            if i in visited:
                continue
            v = i
            visited.add(i)
            if v not in dist or dist[v] > dist[u] + 1:
                dist[v] = dist[u] + 1
                pq.put((dist[v], v))
    
    return dist



def calculate_distances(graph):   
    dist = {}
    for ingredient in graph:
        dist[ingredient] = calculate_distance(graph, ingredient)

    return dist


In [ ]:
dist = calculate_distances(graph) # act as a cache


In [ ]:
#print(dist.keys())

In [ ]:

def fitness(individual):
    counter = 0
    # the higher the worst
    pq = PriorityQueue()
    visited = set()
    for i in range(len(individual['ingredients']) - 1):
        for j in range(i + 1 , len(individual['ingredients'])):
            if i in visited and j in visited:
                continue
            pq.put(dist[individual['ingredients'][i]][individual['ingredients'][j]])

    while not pq.empty():
        u = pq.get()
        counter += u
    return counter

        
print(recipes[4])
print(fitness(recipes[4])) 
print(fitness(recipes[7]))


IndexError: list index out of range